<a href="https://colab.research.google.com/github/JTian-hub/Titanic-Mamdani-Fuzzy-Inference-System/blob/main/Titanic_Mamdani_Fuzzy_System_(by_Tian).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
pip install scikit-fuzzy

In [15]:
# === Box 1: 載入必要套件 ===

import numpy as np
import pandas as pd
import skfuzzy as fuzz
from skfuzzy import control as ctrl

In [16]:
# === Box 2: 定義每個變數的數值範圍（Universe） ===

age_universe    = np.arange(0, 81, 1)
fare_universe   = np.arange(0, 501, 1)
family_universe = np.arange(1, 12, 1)
pclass_universe = np.arange(1, 4, 1)
survival_universe = np.arange(0, 101, 1)

In [17]:
# === Box 3: 定義輸入與輸出變數 ===

Age        = ctrl.Antecedent(age_universe, 'Age')
Fare       = ctrl.Antecedent(fare_universe, 'Fare')
FamilySize = ctrl.Antecedent(family_universe, 'FamilySize')
Pclass     = ctrl.Antecedent(pclass_universe, 'Pclass')

Survival = ctrl.Consequent(survival_universe, 'Survival')


In [18]:
# === Box 4: Age 的 Membership Functions ===

Age['young']  = fuzz.trimf(age_universe, [0, 0, 30])
Age['middle'] = fuzz.trimf(age_universe, [20, 40, 60])
Age['old']    = fuzz.trimf(age_universe, [50, 80, 80])


In [19]:
# === Box 5: Fare 的 Membership Functions ===

Fare['cheap']      = fuzz.trimf(fare_universe, [0, 0, 100])
Fare['medium']     = fuzz.trimf(fare_universe, [50, 200, 350])
Fare['expensive']  = fuzz.trimf(fare_universe, [300, 500, 500])


In [20]:
# === Box 6-1: FamilySize ===

FamilySize['small']  = fuzz.trimf(family_universe, [1, 1, 3])
FamilySize['medium'] = fuzz.trimf(family_universe, [2, 4, 6])
FamilySize['large']  = fuzz.trimf(family_universe, [6, 10, 10])


# === Box 6-2: Pclass ===

Pclass['high'] = fuzz.trimf(pclass_universe, [1, 1, 2])
Pclass['mid']  = fuzz.trimf(pclass_universe, [1, 2, 3])
Pclass['low']  = fuzz.trimf(pclass_universe, [2, 3, 3])


In [21]:
# === Box 7: Survival（輸出） ===

Survival['low']    = fuzz.trimf(survival_universe, [0, 0, 40])
Survival['medium'] = fuzz.trimf(survival_universe, [30, 50, 70])
Survival['high']   = fuzz.trimf(survival_universe, [60, 100, 100])


In [22]:
# === Box 8: Fuzzy Rules（IF–THEN） ===

rule1 = ctrl.Rule(
    Age['young'] & Pclass['high'],
    Survival['high']
)

rule2 = ctrl.Rule(
    Age['old'] & Pclass['low'],
    Survival['low']
)

rule3 = ctrl.Rule(
    Fare['expensive'] & Pclass['high'],
    Survival['high']
)

rule4 = ctrl.Rule(
    FamilySize['large'],
    Survival['low']
)

rule5 = ctrl.Rule(
    Age['middle'] & Fare['medium'],
    Survival['medium']
)


In [23]:
# === Box 9: 建立 Mamdani Control System ===

survival_ctrl = ctrl.ControlSystem([
    rule1, rule2, rule3, rule4, rule5
])

survival_sim = ctrl.ControlSystemSimulation(survival_ctrl)


In [24]:
# === Box 10: 單筆測試 ===

survival_sim.input['Age'] = 25
survival_sim.input['Fare'] = 30
survival_sim.input['FamilySize'] = 1
survival_sim.input['Pclass'] = 1

survival_sim.compute()

print("Survival score:", survival_sim.output['Survival'])


Survival score: 81.61616161616158


In [26]:
# === Box 11: 批次預測 ===

# --- 修正：定義 X_test 和 y_test --- (這是一個示範用的替代方案)
# 在實際應用中，您會在這裡載入您的實際測試資料集並進行預處理。
data = {
    'Age': [22, 38, 26, 35, 35],
    'Fare': [7.25, 71.28, 7.92, 53.10, 8.05],
    'FamilySize': [2, 2, 1, 2, 1],
    'Pclass': [3, 1, 3, 1, 3]
}
X_test = pd.DataFrame(data)
y_test = pd.Series([0, 1, 1, 1, 0]) # 範例生存標籤 (實際應用中需要真實標籤)

def fuzzy_predict(row):
    survival_sim.input['Age'] = row['Age']
    survival_sim.input['Fare'] = row['Fare']
    survival_sim.input['FamilySize'] = row['FamilySize']
    survival_sim.input['Pclass'] = row['Pclass']

    survival_sim.compute()
    # 檢查 'Survival' 是否在輸出中，如果不在，則返回一個中性值
    if 'Survival' in survival_sim.output:
        return survival_sim.output['Survival']
    else:
        return 50.0 # 返回一個中性分數作為預設值

y_score = X_test.apply(fuzzy_predict, axis=1)
y_pred = (y_score >= 50).astype(int)


In [27]:
# === Box 12: 評估結果 ===

from sklearn.metrics import accuracy_score, classification_report

print("Mamdani Fuzzy Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Mamdani Fuzzy Accuracy: 0.4
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.50      0.67      0.57         3

    accuracy                           0.40         5
   macro avg       0.25      0.33      0.29         5
weighted avg       0.30      0.40      0.34         5

